# ChipWhisperer-Lite SimpleSerial Test

This notebook provides example code to:

1. **Verify host/target communication** with the CW-Lite's XMEGA target.
2. **Capture power traces** and perform a basic CPA attack.


In [ ]:
import chipwhisperer as cw

# Connect to scope and target using ChipWhisperer's API
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)

# Example plaintext and key
def tobytes(x):
    return bytes(x)
pt = tobytes(range(16))
key = bytes.fromhex('2b7e151628aed2a6abf7158809cf4f3c')

# Request firmware version
target.write('v\n')
print('Version:', target.read())

# Program key and check for ACK
target.write('k' + key)
print('Key ACK:', target.read())

# Encrypt a test vector and read ciphertext
target.write('p' + pt)
print('Ciphertext:', target.read())

# Clean up
target.dis()
scope.dis()


In [ ]:
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
ktp = cw.ktp.Basic()  # provides random plaintexts
trace_count = 50
traces = []
textins = []

key, _ = ktp.next()  # fixed key for all traces
target.simpleserial_write('k', key)

for _ in range(trace_count):
    _, pt = ktp.next()
    scope.arm()
    target.simpleserial_write('p', pt)
    if scope.capture():
        raise IOError('Capture timeout')
    traces.append(scope.get_last_trace())
    textins.append(pt)

proj = cw.create_project('cw_demo', overwrite=True)
proj.traces.extend(traces, textin=textins, keys=[key]*trace_count)

leak_model = cwa.leakage_models.aes.sbox_output
attack = cwa.cpa(project=proj, leak_model=leak_model)
attack.run()
print('Best key guess:', ''.join('{:02x}'.format(b) for b in attack.best_key))

# Close scope and target
target.dis()
scope.dis()
